<a href="https://colab.research.google.com/github/OVP2023/NN/blob/main/%D0%94%D0%97_DenseNet_161_%D0%90%D1%80%D1%85%D0%B8%D1%82%D0%B5%D0%BA%D1%82%D1%83%D1%80%D1%8B_%D1%81%D0%B2%D0%B5%D1%80%D1%82%D0%BE%D1%87%D0%BD%D1%8B%D1%85_%D1%81%D0%B5%D1%82%D0%B5%D0%B9_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch import nn
import torchvision as tv
from torchsummary import summary
import time
import numpy as np

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [15]:
def evaluate_accuracy(data_iter, net):
    acc_sum, n = 0, 0
    net.eval()
    for X, y in data_iter:
        X, y = X.to(device), y.to(device)
        acc_sum += (net(X).argmax(axis=1) == y).sum()
        n += y.shape[0]
    return acc_sum.item() / n

def train(net, train_iter, test_iter, trainer, num_epochs):
    net.to(device)
    loss = nn.CrossEntropyLoss(reduction='sum')
    net.train()
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()

        for i, (X, y) in enumerate(train_iter):
            X, y = X.to(device), y.to(device)
            trainer.zero_grad()
            y_hat = net(X)
            l = loss(y_hat, y)
            l.backward()
            trainer.step()
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().item()
            n += y.shape[0]

            if i % 10 == 0:
              print(f"Step {i}. time since epoch: {time.time() -  start:.3f}. "
                    f"Train acc: {train_acc_sum / n:.3f}. Train Loss: {train_l_sum / n:.3f}")
            torch.cuda.empty_cache()
        test_acc = evaluate_accuracy(test_iter, net.to(device))
        print('-' * 20)
        print(f'epoch {epoch + 1}, loss {train_l_sum / n:.4f}, train acc {train_acc_sum / n:.3f}'
              f', test acc {test_acc:.3f}, time {time.time() - start:.1f} sec')

In [22]:
transoforms = tv.transforms.Compose([tv.transforms.Grayscale(3), tv.transforms.Resize((224, 224)), tv.transforms.ToTensor()])
Batch_size = 64
train_dataset = tv.datasets.EMNIST('.', split='balanced', train=True, download=True, transform=transoforms)
test_dataset = tv.datasets.EMNIST('.', split='balanced', train=False, download=True, transform=transoforms)

train_data = torch.utils.data.DataLoader(train_dataset, batch_size=Batch_size, shuffle=True)
test_data = torch.utils.data.DataLoader(test_dataset, batch_size=Batch_size)

In [5]:
train_dataset

Dataset EMNIST
    Number of datapoints: 112800
    Root location: .
    Split: Train
    StandardTransform
Transform: Compose(
               Grayscale(num_output_channels=3)
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
           )

In [6]:
print(len(train_dataset.classes))


47


In [27]:
model_densenet161 = tv.models.densenet161()
model_densenet161.fc = nn.Linear(in_features=2048, out_features=47)
optimizer_densenet161 = torch.optim.Adam(model_densenet161.parameters(), lr=0.001)


In [28]:
train(model_densenet161, train_data, test_data, optimizer_densenet161,1)

Step 0. time since epoch: 1.079. Train acc: 0.000. Train Loss: 6.949
Step 10. time since epoch: 5.457. Train acc: 0.034. Train Loss: 5.762
Step 20. time since epoch: 9.804. Train acc: 0.033. Train Loss: 4.995
Step 30. time since epoch: 14.174. Train acc: 0.050. Train Loss: 4.550
Step 40. time since epoch: 18.629. Train acc: 0.058. Train Loss: 4.311
Step 50. time since epoch: 23.037. Train acc: 0.069. Train Loss: 4.119
Step 60. time since epoch: 27.442. Train acc: 0.074. Train Loss: 3.986
Step 70. time since epoch: 31.915. Train acc: 0.084. Train Loss: 3.863
Step 80. time since epoch: 36.335. Train acc: 0.103. Train Loss: 3.726
Step 90. time since epoch: 41.551. Train acc: 0.124. Train Loss: 3.599
Step 100. time since epoch: 46.003. Train acc: 0.129. Train Loss: 3.530
Step 110. time since epoch: 50.458. Train acc: 0.133. Train Loss: 3.468
Step 120. time since epoch: 54.990. Train acc: 0.144. Train Loss: 3.384
Step 130. time since epoch: 59.470. Train acc: 0.163. Train Loss: 3.293
Step 1